In [1]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime
import importlib
import global_settings
importlib.reload(global_settings)
import statsmodels.api as sm
from sklearn.preprocessing import scale

In [2]:
# Define working folder

setts = global_settings.Settings()

root_folder = setts.root_folder
tmp1_folder = setts.tmp1_folder
tmp2_folder = setts.tmp2_folder
output_folder = setts.output_folder

# tmp1_folder = root_folder + "/data/tmp"
# tmp2_folder = root_folder + "/data/tmp2"
# input_folder = root_folder + "/input"
# output_folder = root_folder + "/output"

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
    
year_limit = setts.year_limit

Directory found: D:\Software\temp\CeDX\data_2024_05/data
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp
Directory found: D:\Software\temp\CeDX\data_2024_05/data/tmp2
Directory found: D:\Software\temp\CeDX\data_2024_05/output


In [3]:
# SUA FILE SERVICE TRUOC KHI CHAY CODE
# service = pd.read_csv(root_folder + '/data/service_data.csv')
# service['DOC_DATE'] = service['DOC_DATE'].astype(str).str.replace('-', '')
# service.to_csv(root_folder + '/data/service_data.csv',index=False)


In [4]:
# Read processed service data from code 1
service = pd.read_csv(root_folder + '/data/service_data.csv')
service['DOC_DATE'] = pd.to_datetime(service['DOC_DATE'], format='%Y%m%d', errors='coerce')
service = service[(service['DOC_DATE'] <= setts.month_last_dt)]

# service['DROP_OFF_DATE'] = pd.to_datetime(service['DROP_OFF_DATE'], format='%Y%m%d', errors='coerce')
# service['PICK_UP_DATE'] = pd.to_datetime(service['PICK_UP_DATE'], format='%Y%m%d', errors='coerce')
service['DOC_year'] = service['DOC_DATE'].dt.year.fillna(0)
service['GROSS_VALUE'] = service['GROSS_VALUE'].apply(lambda x: 0 if x < 0 else float(x))
service = service[(service['DOC_year'] > 2015) & (service['DOC_year'] <= year_limit)]
#service = service.drop(['customer_index', 'Cust_group'], axis=1)

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_24448\2219950670.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  service = pd.read_csv(root_folder + '/data/service_data.csv')


In [5]:
# Read processed sales data from code 2
sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')
sale['MODEL'] = sale['MODEL_TEXT_1'].str.split(' ').str[0]
sale = sale.drop('MODEL_TEXT_1', axis=1)
sale['WARRANTY_START_DATE'] = pd.to_datetime(sale['WARRANTY_START_DATE'], format='%Y-%m-%d')
sale['START_year'] = sale['WARRANTY_START_DATE'].dt.year
sale = sale[(sale['START_year'] > 2015) & (sale['START_year'] <= year_limit)]
# sale = sale.drop(['DEALER_CODE', 'customer_index', 'Cust_group'], axis=1)
sale = sale.drop(['DEALER_CODE'], axis=1)

C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_24448\3241315745.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')


In [6]:
service

,ORDER_NUMBER,DEALER_CODE,CUSTOMER_CODE,VIN,DOC_DATE,DROP_OFF_DATE,PICK_UP_DATE,COUNTER_READING,COUNTER_UNIT,PART_NO,...,SALE_UNIT,NET_VALUE,GROSS_VALUE,CUSTOMER_ADVISER,NAME_OF_CUSTOMER_ADVISER,JOB,JOB_TYPE,JOB_TYPE_DESCRIPTION,DESCRIPTION_ONE,DOC_year
778693,3200071114,2020,2000080169,RLHFD1520AY902388,2016-01-02,0,20160101202838,45000,KM,,...,,0,0.0,20205002,Nguyễn Quốc Cường SA,1,1,Periodic Maintenance,"K/T:dầu,nước,gạt mưa,đ/hòa,lốp,ăcquy,đèn",2016
778694,3200071114,2020,2000080169,RLHFD1520AY902388,2016-01-02,0,20160101202838,45000,KM,9410914000,...,ST,3819,4201.0,20205002,Nguyễn Quốc Cường SA,1,1,Periodic Maintenance,Manual_Parts,2016
778695,3200071114,2020,2000080169,RLHFD1520AY902388,2016-01-02,0,20160101202838,45000,KM,08233P99F6NV1,...,ST,408120,448932.0,20205002,Nguyễn Quốc Cường SA,1,1,Periodic Maintenance,Manual_Parts,2016
778696,3200071114,2020,2000080169,RLHFD1520AY902388,2016-01-02,0,20160101202838,45000,KM,X1ENGINE66,...,ST,256300,281930.0,20205002,Nguyễn Quốc Cường SA,1,1,Periodic Maintenance,Manual_Parts,2016
778697,3200071114,2020,2000080169,RLHFD1520AY902388,2016-01-02,0,20160101202838,45000,KM,X1PETROL68,...,ST,246400,271040.0,20205002,Nguyễn Quốc Cường SA,1,1,Periodic Maintenance,Manual_Parts,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19154508,3203800854,1040,2000319110,RLHRS3822PY000179,2024-04-30,0,20240430112300,3,KM,KIỂM TRA PDI,...,STD,240000,259200.0,10405061,Bùi Hải Nam,1,10,PDI,Service,2024
19154509,3203800831,1040,2000319100,RLHRS3821PY000156,2024-04-30,0,20240430112300,3,KM,KIỂM TRA PDI,...,STD,240000,259200.0,10405061,Bùi Hải Nam,1,10,PDI,Service,2024
19154510,3203800763,1040,2000317024,RLHRS3844PY001948,2024-04-30,0,20240430051100,3,KM,KIỂM TRA PDI,...,STD,240000,259200.0,10405061,Bùi Hải Nam,1,10,PDI,Service,2024
19154511,3203800772,2110,2000319087,MRHRV3860RT081340,2024-04-30,20240430020000,20240505010000,3,KM,PDI,...,STD,0,0.0,21105017,TRẦN CÔNG DUY SA,1,10,PDI,PDI,2024


In [7]:
# Join sale and service data based on UID, left join
#service = service.rename(columns={'VIN' : 'UID'})
cb = pd.merge(sale, right=service, how='left', left_on= 'VIN',right_on='VIN')

# Calculate number of years between DOC_DATE and WARRANTY_START_DATE
cb['keika'] = np.ceil((cb['DOC_DATE'] - cb['WARRANTY_START_DATE']).dt.days / 365)

# ?
#cb['keika'] = cb['keika'].replace([np.inf, -np.inf], np.nan)
cb['keika'] = cb['keika'].apply(lambda x: 1 if x == 0 else float(x))
# Warranty = JOB_Type 07
cb['bad_q'] = cb['JOB_TYPE'].apply(lambda x: 0 if pd.isna(x) else (1 if x == 7 or str(x) == '07' or str(x) == '7' else 0))
cb = cb[cb['keika'] > 0]

del sale, service
print(cb.head())

   CUSTOMER_CODE_x  TITLE            CUSTOMER_NAME OCCUPATION  DISTRICT  \
0       2000073677    2.0  VŨ VĂN HỢP+ VŨ VĂN HỢP+        NaN  Biên Hòa   
1       2000073677    2.0  VŨ VĂN HỢP+ VŨ VĂN HỢP+        NaN  Biên Hòa   
2       2000073677    2.0  VŨ VĂN HỢP+ VŨ VĂN HỢP+        NaN  Biên Hòa   
3       2000073677    2.0  VŨ VĂN HỢP+ VŨ VĂN HỢP+        NaN  Biên Hòa   
4       2000073677    2.0  VŨ VĂN HỢP+ VŨ VĂN HỢP+        NaN  Biên Hòa   

                              CUSTOMER_ADDRESS IDENTIFICATION_TYPE  \
0  C4/5 Kp1, P Long Bình, Tp Biên Hòa,đồng Nai              Z00001   
1  C4/5 Kp1, P Long Bình, Tp Biên Hòa,đồng Nai              Z00001   
2  C4/5 Kp1, P Long Bình, Tp Biên Hòa,đồng Nai              Z00001   
3  C4/5 Kp1, P Long Bình, Tp Biên Hòa,đồng Nai              Z00001   
4  C4/5 Kp1, P Long Bình, Tp Biên Hòa,đồng Nai              Z00001   

  IDENTIFICATION_NUMBER  MARITAL_STATUS  PARTNER_CATEGORY  ... GROSS_VALUE  \
0            2000073677             NaN           

In [8]:
# cb.to_csv(output_folder + '/merged_sale_service.csv')

In [9]:
# Calculate tgt v2
tgt = cb[cb['JOB_TYPE_DESCRIPTION'] == 'Periodic Maintenance']
tgt = tgt.groupby('UID').agg({'DOC_DATE': lambda x: len(x.unique()), 'keika': 'max'}).reset_index()
tgt.columns = ['UID', 'PM', 'keika_max']
tgt['freq'] = tgt['PM'] / tgt['keika_max']
tgt = tgt[tgt['freq'] < 11]
tgt['PM_num'] = np.floor(tgt['freq'])
tgt['PM2ov'] = tgt['PM_num'].apply(lambda x: 1 if x >= 2 else 0)
tgt = tgt[['UID', 'PM2ov']]
print(tgt.head())

   UID  PM2ov
0    2      0
1    4      1
2    5      1
3   12      1
4  262      1


In [10]:
# Calculate SA_cont
cb = cb[cb['DOC_year'] < year_limit]
SA_cont = cb[['UID', 'DOC_DATE', 'CUSTOMER_ADVISER']].drop_duplicates()
SA_cont['rank'] = SA_cont.groupby('UID')['DOC_DATE'].rank()
max_rank = SA_cont.groupby('UID')['rank'].transform('max')
SA_cont = SA_cont[SA_cont['rank'] > max_rank - 3]
SA_cont['n'] = SA_cont.groupby('UID')['UID'].transform('count')
SA_cont = SA_cont.groupby(['UID', 'CUSTOMER_ADVISER']).agg({'n': 'count'}).reset_index()
SA_cont.columns = ['UID', 'CUSTOMER_ADVISER', 'count']
SA_cont = SA_cont[SA_cont['count'] > 2]
SA_cont = SA_cont.drop('count', axis=1)
SA_cont['SA_cont'] = 1
print(SA_cont.head())

      UID  CUSTOMER_ADVISER  SA_cont
9      12        30305050.0        1
12    559        30605003.0        1
13    909        31605003.0        1
24   9154        31005010.0        1
30  36297        20205055.0        1


In [11]:
# Calculate JOB_m
JOB_m = cb[cb['JOB_TYPE_DESCRIPTION'].isin(['Body', 'Paint', 'General Repair', 'Periodic Maintenance'])]
JOB_m = JOB_m.groupby(['UID', 'JOB_TYPE_DESCRIPTION']).agg({'GROSS_VALUE': 'sum'}).reset_index()
JOB_m.columns = ['UID', 'JOB_TYPE_DESCRIPTION', 'Total_VALUE']
JOB_m = JOB_m.pivot(index='UID', columns='JOB_TYPE_DESCRIPTION', values='Total_VALUE').reset_index()
JOB_m = JOB_m.fillna(0)
JOB_m['BP'] = JOB_m['Paint'] + JOB_m['Body']
JOB_m = JOB_m.drop(['Paint', 'Body'], axis=1)
JOB_m = JOB_m.rename(columns={'General Repair': 'GR', 'Periodic Maintenance': 'PM'})
print(JOB_m.head())

JOB_TYPE_DESCRIPTION  UID          GR          PM          BP
0                       2  29834346.0   5577289.0   4356000.0
1                       4  18477470.0  12674917.0  28275056.0
2                       5  36016740.0  24705288.0   9235950.0
3                      12   1223000.0   9932832.0   4631220.0
4                     262   1301301.0  15972592.0   1718201.0


In [12]:
# Calculate JOB_vnum
JOB_vnum = cb[cb['JOB_TYPE_DESCRIPTION'].isin(['Body', 'Paint', 'General Repair', 'Periodic Maintenance'])]
JOB_vnum = JOB_vnum.groupby(['UID', 'JOB_TYPE_DESCRIPTION']).agg({'DOC_DATE': lambda x: len(x.unique())}).reset_index()
JOB_vnum.columns = ['UID', 'JOB_TYPE_DESCRIPTION', 'visit_num']
JOB_vnum = JOB_vnum.pivot(index='UID', columns='JOB_TYPE_DESCRIPTION', values='visit_num').reset_index()
JOB_vnum = JOB_vnum.fillna(0)
JOB_vnum['BP_vnum'] = JOB_vnum['Paint'] + JOB_vnum['Body']
JOB_vnum = JOB_vnum.drop(['Paint', 'Body'], axis=1)
JOB_vnum = JOB_vnum.rename(columns={'General Repair': 'GR_vnum', 'Periodic Maintenance': 'PM_vnum'})
print(JOB_vnum.head())

JOB_TYPE_DESCRIPTION  UID  GR_vnum  PM_vnum  BP_vnum
0                       2      4.0      5.0      3.0
1                       4      9.0      9.0      6.0
2                       5     12.0     14.0      5.0
3                      12      7.0     10.0      7.0
4                     262      4.0     12.0      4.0


In [13]:
# Calculate TB
TB = cb.copy()
TB['TIRE'] = TB['DESCRIPTION_ONE'].str.contains('TIRE').astype(int)
TB['BATTERY'] = TB['DESCRIPTION_ONE'].str.contains('BATTERY').astype(int)
TB['OIL'] = ((TB['DESCRIPTION_ONE'].str.contains('OIL')) & (TB['DESCRIPTION_ONE'].str.contains('HONDA'))).astype(int)
TB = TB.groupby('UID').agg({'TIRE': 'sum', 'BATTERY': 'sum', 'OIL': 'sum'}).reset_index()
print(TB.head())

   UID  TIRE  BATTERY  OIL
0    2     0        0    7
1    4     0        1   10
2    5     0        2   17
3   12     0        0    5
4  262     0        0    5


In [14]:
# Calculate PM
PM = cb[cb['JOB_TYPE_DESCRIPTION'] == 'Periodic Maintenance']
PM = PM.groupby('UID').agg({'DOC_year': ['max', lambda x: len(x.unique())], 'START_year': 'min'}).reset_index()
PM.columns = ['UID', 'latest_visit_year', 'visit_year', 'START_year']
PM['visit_rate'] = PM['visit_year'] / (year_limit - PM['START_year'])
PM = PM.drop('START_year', axis=1)
print(PM.head())

   UID  latest_visit_year  visit_year  visit_rate
0    2             2020.0           4    0.500000
1    4             2020.0           4    0.500000
2    5             2023.0           6    0.750000
3   12             2022.0           2    0.666667
4  262             2023.0           5    0.833333


In [15]:
# Calculate smr
smr = cb.groupby('UID').agg({'COUNTER_READING': 'max', 'GROSS_VALUE': 'sum', 'SALES_PRICE': 'sum', 'DOC_DATE': lambda x: len(x.unique()), 'keika': 'max', 'bad_q': 'max', 'START_year': 'min', 'WARRANTY_START_DATE': 'min', 'total_buy': 'max'}).reset_index()
smr.columns = ['UID', 'RUN', 'Total_VALUE', 'SALES_PRICE', 'visit_num', 'keika_max', 'bad_q', 'START_year', 'START_date', 'total_buy']
smr['Total_VALUE'] = smr['Total_VALUE'] + smr['SALES_PRICE']
smr = smr.drop('SALES_PRICE', axis=1)
smr['visit_freq_y'] = smr['visit_num'] / smr['keika_max']
smr = pd.merge(smr, JOB_m, on='UID', how='left')
smr = pd.merge(smr, JOB_vnum, on='UID', how='left')
smr['TTL'] = smr['BP'] + smr['GR'] + smr['PM']
smr['TTL_uni'] = smr['TTL'] / (smr['GR_vnum'] + smr['BP_vnum'] + smr['PM_vnum'])
smr['GR_uni'] = smr['GR'] / smr['GR_vnum']
smr['BP_uni'] = smr['BP'] / smr['BP_vnum']
smr['PM_uni'] = smr['PM'] / smr['PM_vnum']
smr['GR_freq'] = smr['GR_vnum'] / smr['keika_max']
smr['BP_freq'] = smr['BP_vnum'] / smr['keika_max']
smr['PM_freq'] = smr['PM_vnum'] / smr['keika_max']
smr = pd.merge(smr, TB, on='UID', how='left')
smr = pd.merge(smr, PM, on='UID', how='left')
smr = pd.merge(smr, SA_cont, on='UID', how='left')

# Backup smr to check (can be removed)
smr_backup = smr.copy()
print(smr.head())

del cb
del JOB_m
del JOB_vnum
del TB
del PM

   UID      RUN   Total_VALUE  visit_num  keika_max  bad_q  START_year  \
0    2  33265.0  2.616340e+10          9        4.0      0        2016   
1    4  49859.0  4.669960e+10         16        4.0      0        2016   
2    5  56347.0  8.334043e+10         18        7.0      1        2016   
3   12  48717.0  6.261790e+10         15        2.0      0        2021   
4  262  35128.0  3.305081e+10         16        6.0      0        2018   

  START_date  total_buy  visit_freq_y  ...   BP_freq  PM_freq  TIRE  BATTERY  \
0 2016-10-31          1      2.250000  ...  0.750000     1.25     0        0   
1 2016-10-31          1      4.000000  ...  1.500000     2.25     0        1   
2 2016-10-31          1      2.571429  ...  0.714286     2.00     0        2   
3 2021-09-20          2      7.500000  ...  3.500000     5.00     0        0   
4 2018-01-27          2      2.666667  ...  0.666667     2.00     0        0   

   OIL  latest_visit_year  visit_year  visit_rate  CUSTOMER_ADVISER  SA_co

In [16]:
#smr.to_csv(output_folder + '/smr.csv')

In [17]:
# Maintanance Pack
mp = pd.read_csv(tmp1_folder + '/mp.csv')
mp['UID_count'] = mp['UID'].copy()
mp = mp.groupby('UID').agg({'UID_count': 'count'}).reset_index()
mp.columns = ['UID', 'mp_num']
mp['mp'] = 1
print(mp.head())

   UID  mp_num  mp
0    0     109   1
1    2       1   1
2    4       1   1
3    5       1   1
4   98       1   1


In [18]:
# Read tmp2 files calculated from code 3
MH = pd.read_csv(tmp2_folder + '/MH_new_name.csv')

chat = pd.read_csv(tmp2_folder + '/chat_output.csv')

CR = pd.read_csv(tmp2_folder + '/comp_output.csv')

#dcsi
dcsi = pd.read_csv(tmp2_folder + '/DCSI_output_ver2.csv')
dcsi = dcsi[dcsi['Year'] == dcsi.groupby('UID')['Year'].transform('max')]
dcsi.columns = ['UID', 'Month', 'Year', 'Dealer', 'Region', 'satisfaction_reminder', 'satisfaction_reception', 'satisfaction_customer_lounge', 'satisfaction_delivery', 'satisfaction_repair_quality', 'satisfaction_facility', 'one_time_repair', 'total_satisfaction', 'Free_comment', 'follow_call']
dcsi = dcsi.groupby('UID').agg({'total_satisfaction': 'mean', 'satisfaction_reminder': 'mean', 'satisfaction_reception': 'mean', 'satisfaction_customer_lounge': 'mean', 'satisfaction_delivery': 'mean', 'satisfaction_repair_quality': 'mean', 'satisfaction_facility': 'mean', 'one_time_repair': 'mean', 'follow_call': 'mean'}).reset_index()

ew = pd.read_csv(tmp2_folder + '/ew_output.csv')
ew = ew.drop('ew_all_car', axis=1)

eve_v_pro = pd.read_csv(tmp2_folder + '/event_v_promo_new_name.csv')
#eve_v_pro = eve_v_pro.drop('event_v_promo', axis=1)

book = pd.read_csv(tmp2_folder + '/book_new_name.csv')
book = book.drop(['UID_counts_book', 'UID有無'], axis=1)

test_drive = pd.read_csv(tmp2_folder + '/test_drive_output.csv')

estimate_cost = pd.read_csv(tmp2_folder + '/estimate_cost_output.csv')
estimate_cost.columns = ['UID', 'estimate_cost']

view_product_color = pd.read_csv(tmp2_folder + '/view_product_color_output.csv')
view_product_color.columns = ['UID', 'view_product_color']

view_product_detail = pd.read_csv(tmp2_folder + '/view_product_detail_am_output.csv')
view_product_detail.columns = ['UID', 'view_product_detail']

view_product_gallery = pd.read_csv(tmp2_folder + '/view_product_gallery_output.csv')
view_product_gallery.columns = ['UID', 'view_product_gallery']

view_product_list = pd.read_csv(tmp2_folder + '/view_product_list_output.csv')
view_product_list.columns = ['UID', 'view_product_list']

view_promotion_notification = pd.read_csv(tmp2_folder + '/view_promotion_notification_output.csv')

view_test_drive = pd.read_csv(tmp2_folder + '/view_test_drive_output.csv')

sca = pd.read_csv(tmp2_folder + '/SCA_output.csv')

notif_book_pi = pd.read_csv(tmp2_folder + '/notif_book_pi_new_name.csv')

notif_book_pm = pd.read_csv(tmp2_folder + '/notif_book_pm_new_name.csv')

notif_book_spa = pd.read_csv(tmp2_folder + '/notif_book_spa_new_name.csv')

notif_v_pro = pd.read_csv(tmp2_folder + '/notif_v_pro_new_name.csv')

notif_v_spa = pd.read_csv(tmp2_folder + '/notif_v_spa_new_name.csv')

notif_vpi = pd.read_csv(tmp2_folder + '/notif_vpi_new_name.csv')

notif_vpm = pd.read_csv(tmp2_folder + '/notif_vpm_new_name.csv')

print("All files read")

All files read


In [19]:
mp['UID'] = mp['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
MH['UID'] = MH['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
chat['UID'] = chat['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
CR['UID'] = CR['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
dcsi['UID'] = dcsi['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
ew['UID'] = ew['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
eve_v_pro['UID'] = eve_v_pro['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
book['UID'] = book['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
test_drive['UID'] = test_drive['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
estimate_cost['UID'] = estimate_cost['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_color['UID'] = view_product_color['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_detail['UID'] = view_product_detail['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_gallery['UID'] = view_product_gallery['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_product_list['UID'] = view_product_list['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_promotion_notification['UID'] = view_promotion_notification['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
view_test_drive['UID'] = view_test_drive['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
sca['UID'] = sca['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_book_pi['UID'] = notif_book_pi['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_book_pm['UID'] = notif_book_pm['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_book_spa['UID'] = notif_book_spa['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_v_pro['UID'] = notif_v_pro['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_v_spa['UID'] = notif_v_spa['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_vpi['UID'] = notif_vpi['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_vpm['UID'] = notif_vpm['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
notif_vpm['UID'] = notif_vpm['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
tgt['UID'] = tgt['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
print("Cleaned UID fields for data")

Cleaned UID fields for data


In [20]:
# Load smr backup to test
smr = smr_backup.copy()
smr['UID'] = smr['UID'].astype(str).apply(lambda x: x.replace(".0", ""))
print(smr.head())

   UID      RUN   Total_VALUE  visit_num  keika_max  bad_q  START_year  \
0    2  33265.0  2.616340e+10          9        4.0      0        2016   
1    4  49859.0  4.669960e+10         16        4.0      0        2016   
2    5  56347.0  8.334043e+10         18        7.0      1        2016   
3   12  48717.0  6.261790e+10         15        2.0      0        2021   
4  262  35128.0  3.305081e+10         16        6.0      0        2018   

  START_date  total_buy  visit_freq_y  ...   BP_freq  PM_freq  TIRE  BATTERY  \
0 2016-10-31          1      2.250000  ...  0.750000     1.25     0        0   
1 2016-10-31          1      4.000000  ...  1.500000     2.25     0        1   
2 2016-10-31          1      2.571429  ...  0.714286     2.00     0        2   
3 2021-09-20          2      7.500000  ...  3.500000     5.00     0        0   
4 2018-01-27          2      2.666667  ...  0.666667     2.00     0        0   

   OIL  latest_visit_year  visit_year  visit_rate  CUSTOMER_ADVISER  SA_co

In [21]:
# Merge smr with other file
smr = pd.merge(smr, mp, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("mp", len(smr)))
smr = pd.merge(smr, MH, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("MH", len(smr)))
smr = pd.merge(smr, chat, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("chat", len(smr)))
smr = pd.merge(smr, CR, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("CR", len(smr)))
smr = pd.merge(smr, dcsi, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("dcsi", len(smr)))
smr = pd.merge(smr, ew, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("ew", len(smr)))
smr = pd.merge(smr, eve_v_pro, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("eve_v_pro", len(smr)))
smr = pd.merge(smr, book, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("book", len(smr)))
smr = pd.merge(smr, test_drive, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("test_drive", len(smr)))
smr = pd.merge(smr, estimate_cost, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("estimate_cost", len(smr)))
smr = pd.merge(smr, view_product_color, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_color", len(smr)))
smr = pd.merge(smr, view_product_detail, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_detail", len(smr)))
smr = pd.merge(smr, view_product_gallery, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_gallery", len(smr)))
smr = pd.merge(smr, view_product_list, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_product_list", len(smr)))
smr = pd.merge(smr, view_promotion_notification, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_promotion_notification", len(smr)))
smr = pd.merge(smr, view_test_drive, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("view_test_drive", len(smr)))
smr = pd.merge(smr, sca, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("sca", len(smr)))
smr = pd.merge(smr, notif_book_pi, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_book_pi", len(smr)))
smr = pd.merge(smr, notif_book_pm, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_book_pm", len(smr)))
smr = pd.merge(smr, notif_book_spa, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_book_spa", len(smr)))
smr = pd.merge(smr, notif_v_pro, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_v_pro", len(smr)))
smr = pd.merge(smr, notif_v_spa, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_v_spa", len(smr)))
smr = pd.merge(smr, notif_vpi, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_vpi", len(smr)))
smr = pd.merge(smr, notif_vpm, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("notif_vpm", len(smr)))
smr = pd.merge(smr, tgt, on="UID", how='left')
print("Merged with: %s  - column count: %s" % ("tgt", len(smr)))


print("Data merged")

Merged with: mp  - column count: 153851
Merged with: MH  - column count: 153851
Merged with: chat  - column count: 153851
Merged with: CR  - column count: 153851
Merged with: dcsi  - column count: 153851
Merged with: ew  - column count: 153851
Merged with: eve_v_pro  - column count: 153851
Merged with: book  - column count: 153851
Merged with: test_drive  - column count: 153851
Merged with: estimate_cost  - column count: 153851
Merged with: view_product_color  - column count: 153851
Merged with: view_product_detail  - column count: 153851
Merged with: view_product_gallery  - column count: 153851
Merged with: view_product_list  - column count: 153851
Merged with: view_promotion_notification  - column count: 153851
Merged with: view_test_drive  - column count: 153851
Merged with: sca  - column count: 153851
Merged with: notif_book_pi  - column count: 153851
Merged with: notif_book_pm  - column count: 153851
Merged with: notif_book_spa  - column count: 153851
Merged with: notif_v_pro  - c

In [22]:
smr = smr.assign(visit_rate=lambda x: x['visit_rate'].apply(lambda y: 100 if y > 1 else y*100))
smr = smr.assign(keika_max=lambda x: year_limit+1-x['START_year'])
smr = smr[smr['keika_max'] > 0]
#smr = smr.drop(columns=['PM_freq', 'BP_freq', 'GR_freq', 'visit_year'])
#smr.drop(columns = ['CUSTOMER_ADVISER', 'visit_num', 'visit_freq_y', 'GR_vnum', 'PM_vnum', 'BP_vnum', 'TTL_uni', 'TTL', 'GR_uni', 'BP_uni', 'PM_uni', 'RUN', 'OIL', 'Total_VALUE', 'GR', 'PM', 'BP', 'ew_sales', 'app_mente_SA_select', 'app_mente_SA_select', 'CR_feedback', 'satisfaction_mechanic', 'mp_num', 'bad_q', 'notif_book_pi', 'notif_book_pm', 'notif_book_spa', 'test_drive', 'follow_call', 'one_time_repair', 'coupon_view', 'view_product_color', 'view_product_gallery', 'chat_num', 'sca_visit_num', 'view_product_list', 'view_promotion_notification', 'MH_Active', 'latest_visit_year', 'service_part'], axis=1, inplace=True)

drop_list = [col for col in smr.columns if col.startswith('satisfaction')]
drop_list.extend(['PM_freq', 'BP_freq', 'GR_freq', 'visit_year'])
drop_list.extend(['CUSTOMER_ADVISER', 'visit_num', 'visit_freq_y', 'GR_vnum', 'PM_vnum', 'BP_vnum', 'TTL_uni', 'TTL', 'GR_uni', 'BP_uni', 'PM_uni', 'RUN', 'OIL', 'Total_VALUE', 'GR', 'PM', 'BP', 'ew_sales', 'app_mente_SA_select', 'app_mente_SA_select', 'CR_feedback', 'satisfaction_mechanic', 'mp_num', 'bad_q', 'notif_book_pi', 'notif_book_pm', 'notif_book_spa', 'test_drive', 'follow_call', 'one_time_repair', 'coupon_view', 'view_product_color', 'view_product_gallery', 'chat_num', 'sca_visit_num', 'view_product_list', 'view_promotion_notification', 'MH_Active', 'latest_visit_year', 'service_part'])
drop_list.extend([col for col in smr.columns if col.startswith('UID有無')])
cols_to_drop = set(smr.columns).intersection(drop_list)

#smr = smr.drop(columns=cols_to_drop)
del mp, MH, chat, CR, dcsi, ew, eve_v_pro, book, test_drive, estimate_cost, view_product_color, view_product_detail, view_product_gallery, view_product_list, view_promotion_notification, view_test_drive, sca, notif_book_pi, notif_book_pm, notif_book_spa, notif_v_pro, notif_v_spa, notif_vpi, notif_vpm
print(smr.head())

   UID      RUN   Total_VALUE  visit_num  keika_max  bad_q  START_year  \
0    2  33265.0  2.616340e+10          9          9      0        2016   
1    4  49859.0  4.669960e+10         16          9      0        2016   
2    5  56347.0  8.334043e+10         18          9      1        2016   
3   12  48717.0  6.261790e+10         15          4      0        2021   
4  262  35128.0  3.305081e+10         16          7      0        2018   

  START_date  total_buy  visit_freq_y  ...  view_test_drive  sca_visit_num  \
0 2016-10-31          1      2.250000  ...              NaN            NaN   
1 2016-10-31          1      4.000000  ...              NaN            NaN   
2 2016-10-31          1      2.571429  ...              NaN            NaN   
3 2021-09-20          2      7.500000  ...              NaN            NaN   
4 2018-01-27          2      2.666667  ...              NaN            NaN   

   notif_book_pi  notif_book_pm  notif_book_spa  coupon_view  notif_v_spa  \
0        

In [23]:
# Export merged data
data = smr[smr['keika_max'] > 0]
field_backup = data[['UID', 'START_date', 'START_year']]
data = data.drop(['UID', 'START_date', 'START_year'], axis=1)
data = data.apply(pd.to_numeric).fillna(0)
data = pd.concat([field_backup, data], axis=1)
data = data[data['UID'] != 0]
data = data[data['UID'] != '0']
data = data.rename(columns={'MyHondaæœ‰ç„¡' : 'MH_ID', 'MyHonda有無' : 'MH_ID'})

output_data_folder = output_folder + "/dataset_all_full_columns.csv"
data.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)
print(data.columns)
data_backup = data.copy()

Export data to D:\Software\temp\CeDX\data_2024_05/output/dataset_all_full_columns.csv
Index(['UID', 'START_date', 'START_year', 'RUN', 'Total_VALUE', 'visit_num',
       'keika_max', 'bad_q', 'total_buy', 'visit_freq_y', 'GR', 'PM', 'BP',
       'GR_vnum', 'PM_vnum', 'BP_vnum', 'TTL', 'TTL_uni', 'GR_uni', 'BP_uni',
       'PM_uni', 'GR_freq', 'BP_freq', 'PM_freq', 'TIRE', 'BATTERY', 'OIL',
       'latest_visit_year', 'visit_year', 'visit_rate', 'CUSTOMER_ADVISER',
       'SA_cont', 'mp_num', 'mp', 'MH_Active', 'satisfaction_service',
       'satisfaction_mechanic', 'satisfaction_chatbot',
       'satisfaction_chatbot_agent', 'MH_ID', 'chat_num', 'service_part',
       'chat', 'CR_feedback', 'CR', 'total_satisfaction',
       'satisfaction_reminder', 'satisfaction_reception',
       'satisfaction_customer_lounge', 'satisfaction_delivery',
       'satisfaction_repair_quality', 'satisfaction_facility',
       'one_time_repair', 'follow_call', 'ew_sales', 'ew_num', 'event_v_promo',
       

In [24]:
# print(data_backup.tail())
# output_data_folder = output_folder + "/dataset_all_full_columns test.csv"
# data_backup['UID'].to_csv(output_data_folder, index=False)

In [25]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

year_limit = setts.year_limit
end_date_time = setts.month_to
print('Reading service file')
service = pd.read_csv(root_folder + '/data/service_data.csv')
service['DOC_DATE'] = pd.to_datetime(service['DOC_DATE'], format='%Y%m%d', errors='coerce')
service['DROP_OFF_DATE'] = pd.to_datetime(service['DROP_OFF_DATE'], format='%Y%m%d', errors='coerce')
service['PICK_UP_DATE'] = pd.to_datetime(service['PICK_UP_DATE'], format='%Y%m%d', errors='coerce')
service['DOC_year'] = service['DOC_DATE'].dt.year.fillna(0)
service['GROSS_VALUE'] = service['GROSS_VALUE'].apply(lambda x: 0 if x < 0 else float(x))
service = service[(service['DOC_year'] > 2015) & (service['DOC_year'] <= year_limit)]
# service = service.drop(['customer_index', 'Cust_group'], axis=1)

print('Reading sale file')
sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')
sale['MODEL'] = sale['MODEL_TEXT_1'].str.split(' ').str[0]
sale = sale.drop('MODEL_TEXT_1', axis=1)
sale['WARRANTY_START_DATE'] = pd.to_datetime(sale['WARRANTY_START_DATE'], format='%Y-%m-%d')
sale['START_year'] = sale['WARRANTY_START_DATE'].dt.year
sale = sale[(sale['START_year'] > 2015) & (sale['START_year'] <= year_limit)]
sale = sale.drop(['DEALER_CODE'], axis=1)

print('Merging files')

df = pd.merge(sale, service, on='VIN', how='left')
del sale, service

df = df[df['DOC_year'] <= year_limit]
df['keika'] = np.ceil((df['DOC_DATE'] - df['WARRANTY_START_DATE']).dt.days / 365)
df['keika'] = df['keika'].apply(lambda x: 1 if x == 0 else float(x))
df = df[df['keika'] > 0]


df = df[['UID', 'DOC_DATE', 'JOB_TYPE_DESCRIPTION']]
# df = df[df['JOB_TYPE_DESCRIPTION'] == 'Periodic Maintenance']
df = df.sort_values(by=['DOC_DATE'])
df['DOC_DATE'] = df['DOC_DATE'].dt.strftime('%Y%m%d').astype(int)
df = df[df['DOC_DATE'] <= end_date_time]
df = df.drop_duplicates(subset=['UID'], keep='last')
df = df.sort_values(by=['UID'])

df['DOC_DATE'] = pd.to_datetime(df['DOC_DATE'], format='%Y%m%d')
end_date = pd.to_datetime(str(end_date_time), format='%Y%m%d')
df['month_to_' + str(end_date_time)] = (end_date.year - df['DOC_DATE'].dt.year) * 12 + (end_date.month - df['DOC_DATE'].dt.month)

df = df[df['UID'] != 0]
df = df[df['UID'] != '0']

print('Merging with dataset_all_full_columns')


output_data_folder = output_folder + "/dataset_all_full_columns.csv"
dataset_full = pd.read_csv(output_data_folder)
dataset_full = pd.merge(dataset_full, df, on='UID', how='left')

dataset_full.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)


Reading service file


C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_24448\1082347448.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  service = pd.read_csv(root_folder + '/data/service_data.csv')


Reading sale file


C:\Users\AO_HVN_ITS_LABO_DWH\AppData\Local\Temp\ipykernel_24448\1082347448.py:17: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sale = pd.read_csv(tmp1_folder + '/sale_uni_min.csv')


Merging files
Merging with dataset_all_full_columns
Export data to D:\Software\temp\CeDX\data_2024_05/output/dataset_all_full_columns.csv


In [28]:
# print(dataset_full.tail())
# output_data_folder = output_folder + "/dataset_all_full_columns test2.csv"
# dataset_full.to_csv(output_data_folder, index=False)

In [27]:
# Only keep relevant columns
data_shrink = data_backup.copy()
data_shrink = data_shrink[['UID', 'keika_max', 'total_buy', 'TIRE', 'BATTERY', 'visit_rate', 'SA_cont', 'mp', 'MH_ID', 'chat', 'CR', 'total_satisfaction', 'ew_num', 'coupon_use', 'SA_select', 'app_reserve_mean', 'estimate_cost', 'view_product_detail', 'view_test_drive', 'notif_v_spa', 'notif_vpi', 'notif_vpm', 'PM2ov']]
output_data_folder = output_folder + "/dataset_all.csv"
data_shrink.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)
print(data_shrink.columns)
data_shrink_backup = data_shrink.copy()

Export data to D:\Software\temp\CeDX\data_2024_05/output/dataset_all.csv
Index(['UID', 'keika_max', 'total_buy', 'TIRE', 'BATTERY', 'visit_rate',
       'SA_cont', 'mp', 'MH_ID', 'chat', 'CR', 'total_satisfaction', 'ew_num',
       'coupon_use', 'SA_select', 'app_reserve_mean', 'estimate_cost',
       'view_product_detail', 'view_test_drive', 'notif_v_spa', 'notif_vpi',
       'notif_vpm', 'PM2ov'],
      dtype='object')


# Apply ML to create model

In [35]:
data = data_shrink_backup.copy()
PM2ov = data['PM2ov']
data = data.drop('PM2ov', axis=1)
print(pd.Series(PM2ov).value_counts())
print(data.shape)
print(data.head())

0.0    85605
1.0    68246
Name: PM2ov, dtype: int64
(153851, 22)
   UID  keika_max  total_buy  TIRE  BATTERY  visit_rate  SA_cont   mp  MH_ID  \
0    2          9          1     0        0   50.000000      0.0  1.0    0.0   
1    4          9          1     0        1   50.000000      0.0  1.0    0.0   
2    5          9          1     0        2   75.000000      0.0  1.0    0.0   
3   12          4          2     0        0   66.666667      1.0  0.0    0.0   
4  262          7          2     0        0   83.333333      0.0  0.0    0.0   

   chat  ...  ew_num  coupon_use  SA_select  app_reserve_mean  estimate_cost  \
0   0.0  ...     0.0         0.0        0.0               0.0            0.0   
1   0.0  ...     0.0         0.0        0.0               0.0            0.0   
2   0.0  ...     0.0         0.0        1.0              12.0            0.0   
3   0.0  ...     0.0         0.0        0.0               0.0            0.0   
4   1.0  ...     1.0         0.0        0.0           

In [36]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

data = data_shrink_backup.copy()
scaler = StandardScaler()
sc = scaler.fit_transform(data)
z = pd.DataFrame(sc, index=data.index, columns=data.columns)
output_data_folder = output_folder + "/dataset_all_scaled.csv"
z.to_csv(output_data_folder, index=False)
print("Export data to", output_data_folder)


Export data to D:\Software\temp\CeDX\data_2024_05/output/dataset_all_scaled.csv


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
sc = scaler.fit_transform(data)
z = pd.DataFrame(sc, index=data.index, columns=data.columns)
z['PM2ov'] = PM2ov

X = z.drop('PM2ov', axis=1)
y = z['PM2ov']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

roc_auc = roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1])
print(f"ROC AUC Score: {roc_auc:.4f}")

data_out = data_shrink_backup.copy()
data_out['Score'] = rf_model.predict_proba(X)[:, 1]
output_data_folder = output_folder + "/dataset_all_scored_rf.csv"
data_out.to_csv(output_data_folder, index=False)

Accuracy: 0.6554
Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.68      0.69     17068
         1.0       0.61      0.62      0.62     13703

    accuracy                           0.66     30771
   macro avg       0.65      0.65      0.65     30771
weighted avg       0.66      0.66      0.66     30771

ROC AUC Score: 0.7145


In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy after tuning: {accuracy:.4f}")


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


C:\ProgramData\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found:  {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy after tuning: 0.7150


In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore")  # Ignore warnings for cleaner output

# ================================
# 1️⃣ Load Data (Replace with Your Dataset)
# ================================
# Assuming X and y are already defined
# X = df.drop(columns=['target'])  # Features
# y = df['target']  # Target variable


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ================================
# 2️⃣ Feature Selection Using p-value (ANOVA F-test)
# ================================
selector = SelectKBest(score_func=f_classif, k=10)  # Select top 10 features
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Get p-values
p_values = selector.pvalues_
selected_features = np.argsort(p_values)[:10]  # Select features with lowest p-values

# Print selected features
print("Selected Features Based on p-values:")
for i in selected_features:
    print(f"Feature {i}, p-value: {p_values[i]:.4f}")

# ================================
# 3️⃣ Apply Polynomial Features (Degree 2)
# ================================
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_selected)
X_test_poly = poly.transform(X_test_selected)

# ================================
# 4️⃣ Train Random Forest with Hyperparameter Tuning
# ================================
rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=rf_model, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs=-1, 
    verbose=2
)

grid_search.fit(X_train_poly, y_train)

# ================================
# 5️⃣ Evaluate Best Model
# ================================
print("Best parameters found: ", grid_search.best_params_)

best_rf_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_rf_model.predict(X_test_poly)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy after feature selection & tuning: {accuracy:.4f}")


Selected Features Based on p-values:
Feature 3, p-value: 0.0000
Feature 4, p-value: 0.0000
Feature 5, p-value: 0.0000
Feature 6, p-value: 0.0000
Feature 14, p-value: 0.0000
Feature 15, p-value: 0.0000
Feature 9, p-value: 0.0000
Feature 8, p-value: 0.0000
Feature 7, p-value: 0.0000
Feature 11, p-value: 0.0000
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy after feature selection & tuning: 0.7153


In [44]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train_poly, y_train)
y_pred_base = logreg.predict(X_test_poly)
baseline_acc = accuracy_score(y_test, y_pred_base)

print(f"Baseline Logistic Regression Accuracy: {baseline_acc:.4f}")


Baseline Logistic Regression Accuracy: 0.7134


In [39]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_train)

# Train the Random Forest model with polynomial features
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_poly, y_train)

# Predict on the test data
X_test_poly = poly.transform(X_test)
y_pred = rf_model.predict(X_test_poly)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with polynomial features: {accuracy:.4f}")


Accuracy with polynomial features: 0.6635


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_train)

# Train the Random Forest model with polynomial features
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_poly, y_train)

# Predict on the test data
X_test_poly = poly.transform(X_test)
y_pred = rf_model.predict(X_test_poly)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with polynomial features: {accuracy:.4f}")


In [31]:
# import statsmodels.api as sm
# from sklearn.preprocessing import StandardScaler

# data = data_shrink_backup.copy()
# #data = data.drop('UID', axis=1)
# scaler = StandardScaler()
# sc = scaler.fit_transform(data)

# z = pd.DataFrame(sc, index=data.index, columns=data.columns)
# z['PM2ov'] = PM2ov
# X = z.drop('PM2ov', axis=1)
# y = z['PM2ov']

# print(z)
# formula = "PM2ov~keika_max + total_buy + TIRE + BATTERY + visit_rate + mp + MH_ID + chat + CR + total_satisfaction + ew_num + coupon_use + SA_select + app_reserve_mean + estimate_cost + view_product_detail + view_test_drive + notif_v_spa + notif_vpi + notif_vpm + SA_cont + visit_rate:keika_max+SA_select:SA_cont"
# #formula = "PM2ov ~ keika_max + TIRE + BATTERY + visit_rate + mp + chat + CR + ew_num + coupon_use + SA_select + app_reserve_mean + notif_vpm + SA_cont + keika_max:visit_rate"
# model = sm.formula.glm(formula = formula, data = z, family = sm.families.Binomial()).fit()
# best_model = model
# while True:
#     pvalues = best_model.pvalues.drop('Intercept')
#     if pvalues.max() > 0.05:
#         worst_feature = pvalues.idxmax()
#         print("Removed %s" % (worst_feature))
#         new_formula = formula.replace(f" + {worst_feature}", "")
#         #new_formula = new_formula.replace(f" {worst_feature}:", "")
#         #new_formula = new_formula.replace(f":{worst_feature} ", "")
#         new_model = sm.formula.glm(formula=new_formula, data=data, family=sm.families.Binomial()).fit()
#         if new_model.aic < best_model.aic:
#             best_model = new_model
#             formula = new_formula
#         else:
#             break
#     else:
#         break
# # View the final model summary
# print(model.aic)
# print(formula)
# print(model.summary())
# data_out = data_shrink_backup.copy()
# data_out['Score'] = model.fittedvalues
# output_data_folder = output_folder + "/dataset_all_scored.csv"
# data_out.to_csv(output_data_folder, index=False)


               UID  keika_max  total_buy      TIRE   BATTERY  visit_rate  \
0        -0.133491   1.936144  -0.105263 -0.247642 -0.497694   -0.530917   
1        -0.133491   1.936144  -0.105263 -0.247642  0.932332   -0.530917   
2        -0.133491   1.936144  -0.105263 -0.247642  2.362359    0.221530   
3        -0.133491  -0.482794   2.490072 -0.247642 -0.497694   -0.029286   
4        -0.133491   0.968569   2.490072 -0.247642 -0.497694    0.472345   
...            ...        ...        ...       ...       ...         ...   
153846  124.628222   1.936144  -0.105263 -0.247642  2.362359   -0.154694   
153847  126.190188   0.484781  -0.105263 -0.247642  0.932332    0.372019   
153848  131.271296   0.968569  -0.105263 -0.247642 -0.497694    0.472345   
153849  131.516259   1.936144  -0.105263 -0.247642  0.932332    0.221530   
153850  131.724065   1.936144  -0.105263 -0.247642 -0.497694   -0.907141   

         SA_cont        mp     MH_ID      chat  ...  coupon_use  SA_select  \
0      -0

## Only run code below to test HM's dataset_all

In [33]:
# Save model after creating it
model.save(output_folder + '/customer_engagement_index_model.pkl')

In [34]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


model_path = output_folder + "/customer_engagement_index_model.pkl"

# load the saved model from file
model = sm.load(model_path)
#print(model.summary())
#print(dir(model.params))
print(model.params)

Intercept              -0.365445
keika_max              -0.063252
total_buy               0.003367
TIRE                    0.151353
BATTERY                 0.159683
visit_rate              1.069068
mp                      0.078041
MH_ID                   0.046397
chat                    0.049824
CR                     -0.087488
total_satisfaction      0.038274
ew_num                  0.033613
coupon_use              0.024102
SA_select               0.062005
app_reserve_mean        0.056309
estimate_cost           0.012452
view_product_detail    -0.005127
view_test_drive        -0.002808
notif_v_spa             0.006080
notif_vpi              -0.002160
notif_vpm              -0.109962
SA_cont                 0.167446
visit_rate:keika_max   -0.139780
SA_select:SA_cont       0.007342
dtype: float64
